In [5]:
#imports
import matplotlib.pyplot as plt  # Importing matplotlib for plots
import numpy as np  # arrays

plt.rcParams.update({'font.size': 14})
import IPython.display as ipd  # To play signals
import sounddevice as sd
import soundfile as sf
#Os proximos modulos são usados para crir o nosso SLIT
#from scipy.signal import butter, Lfilter, freqs, tf2sos, sosfilt
from scipy import signal

ModuleNotFoundError: No module named 'soundfile'

In [ ]:
#import do sinal para filtragem
#https://freesound.org/people/augustoschwartz/sounds/251255/

xn, fs = sf.read('251255__augustoschwartz__miguilim.mp3')
print(fs)
xn = xn[:,0] #only one channel
time_xn = np.linspace(0, (len(xn)-1)/fs, len(xn))
#time_xn = np.linspace(0, (len(xn)-1/fs,len(xn)))

plt.figure(figsize=(12,3))
plt.plot(time_xn, xn, '-k', linewidth=1, alpha=0.8)
plt.grid(linestyle = '--', which='both')
plt.xlabel('Tempo [s]')
plt.ylabel(r'$x(t)$ [-]')
plt.xlim(0, time_xn[-1])
plt.tight_layout()
plt.show()

ipd.Audio(xn, rate=fs) #load a NumPy array

NameError: name 'sf' is not defined

In [ ]:
#IIR FILTER SPECS
filter_specs = {
    'order': 2,
    'cutoff_freq': 1000,
    'type': 'hp',
    'sampling_freq': fs
}

In [ ]:
#Architecture for actually filtering with scipy
sos = signal.butter(N = filter_specs['order'], Wn=filter_specs['cutoff_freq'],
                    btype=filter_specs['type'], fs=filter_specs['sampling_freq'],
                    output='sos')

#zeros, poles and gain
z, p, k = signal.sos2zpk(sos)

#a and b coefficients
b, a = signal.zpk2tf(z, p, k)

# Transfer function
freq, H = signal.sosfreqz(sos, 4100, fs=fs)

# Group Delay
freqg, gd = signal.group_delay((b,a), 512, fs=fs)

fig = plt.figure(figsize=(8, 7))

# Subplot 1
ax1 = fig.add_subplot(2, 2, 1)
ax1.semilogx(freq, 20 * np.log10(np.maximum(abs(H), 1e-5)))
ax1.set_xlabel('Frequency [Hz]')
ax1.set_ylabel(r'$|H(j \omega)|$ [dB]')
ax1.axis((10, fs / 2, -100, 10))
ax1.grid(which='both', axis='both')

# Subplot 2
ax2 = fig.add_subplot(2, 2, 2)
ax2.semilogx(freq, np.rad2deg(np.angle(H)))
ax2.set_xlabel('Frequency [Hz]')
ax2.set_ylabel(r'$\angle H(j \omega)$ [$^\circ$]')
ax2.axis((10, fs / 2, -200, 200))
ax2.grid(which='both', axis='both')

# Subplot 3
ax3 = fig.add_subplot(2, 2, 3)
ax3.semilogx(freqg, 1000 * gd / fs)
ax3.set_xlabel('Frequency [Hz]')
ax3.set_ylabel(r'$-\frac{d\angle H(j \omega)}{d \omega}$ [ms]')
ax3.axis((10, fs / 2, 0, 0.3))
ax3.grid(which='both', axis='both')

# Subplot 4
ax4 = fig.add_subplot(2, 2, 4)
ax4.scatter(np.real(p), np.imag(p), color='k', marker='x', s=100)
ax4.scatter(np.real(z), np.imag(z), color='r', marker='o', s=100)
theta = np.linspace(0, 2 * np.pi, 500)
ax4.plot(np.cos(theta), np.sin(theta), '--k')
ax4.grid(which='both', axis='both')
ax4.set_xlabel('Re{z}')
ax4.set_ylabel('Im{z}')

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

In [6]:
#FILTER
xn_filtered = signal.sosfilt(sos,xn)

#plot
plt.figure(figsize=(12,3))
plt.plot(time_xn, xn_filtered, linewidth=1, alpha=0.8,
         label='Filtered Signal')
plt.legend(loc='upper right')
plt.grid(linestyle = '--', which='both')
plt.xlabel('Tempo [s]')
plt.ylabel(r'$x(t)$ [-]')
plt.xlim(0, time_xn[-1])
plt.tight_layout()

plt.figure(figsize=(12,3))
plt.plot(time_xn, xn, '-k', linewidth=1, alpha=0.8,
         label='Original Signal')
plt.legend(loc='upper right')
plt.grid(linestyle = '--', which='both')
plt.xlabel('Tempo [s]')
plt.ylabel(r'$x(t)$ [-]')
plt.xlim(0, time_xn[-1])
plt.tight_layout()

ipd.Audio(xn_filtered, rate=fs)

NameError: name 'signal' is not defined

In [ ]:
from tqdm.notebook import trange, tqdm

def our_filter(xn, b, a):
    
    #length of xn
    N = len(xn)
    #order
    od = len(b)
    #initialize yn
    yn = np.zeros(N)
    #loop
    #bar = tqdm(total=N, desc= 'Filtering ...')
    
    for jn in np.arange(od, N-od): #range(od, N):
        x_slice = np.flip(xn[jn - od : jn])
        y_slice = np.flip(yn[jn - od + 1 : jn+1])
        yn[jn] = np.dot(b, x_slice) - np.dot(a, y_slice)
        #bar.update(1)
    #bar.close()
    gain = np.std(xn) / np.std(yn)
        
    #bar.close()
    #gain = np.std(xn)/np.std(yn)
    return yn
    

In [ ]:
yn = our_filter(xn, b, a)

#plot here yn and xn if you desire.

ipd.Audio(yn, rate=fs)

In [ ]:
#FILTRO FIR

# Define Nyquist frequency
nyquist = 0.5 * fs
#step = 200

#bands = np.arange(0,nyquist+step,step)
#desired = np.ones_like(bands)
#desired[bands <= 1000] = 0
#desired[bands >= 2000] = 0

# Define desired frequency bands and corresponding gains
#bands = [0, 1, 2, 5]  # Frequency bands in Hz
#desired = [1, 1, 0, 0]  # Desired gains for each band (1 for passband, 0 for stopband)

#bands = [0, 1000, 1001, 1999, 2000, nyquist]  # Frequency bands in Hz 
bands = [0, 1000, 1001, 1999, 2000, 3999, 4000, 5000, 5200, nyquist]
desired = [0, 0, 1, 1, 0, 0, 1, 1, 0, 0]  # Desired gains for each band (1 for passband, 0 for stopband)

fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(1,1,1)
ax.plot(bands, desired, color='r')
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel(r'$|H(j \omega)|$')
ax.axis((100, nyquist, 0, 1.2))
ax.grid(which='both', axis='both')

# Normalize frequency bands to Nyquist frequency
bands = [band / nyquist for band in bands]

# Combine passbands and stopbands into pairs
bands = [bands[i:i+2] for i in range(0, len(bands), 2)]

# Specify the filter order (number of taps)
numtaps = 65  # You can adjust this value as needed

# Use the signal.firls function to design the filter
coefficients = signal.firls(numtaps, bands, desired)

# Plot the frequency response of the designed filter
w, h = signal.freqz(coefficients)
plt.plot(0.5 * fs * w / np.pi, np.abs(h), 'b')
plt.title('Frequency response')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Gain')
plt.grid()
plt.show()

sos_f = signal.tf2sos(coefficients, 1, pairing='nearest')

# Zeros, poles, and gain
z_f, p_f, k_f = signal.sos2zpk(sos_f)

# Transfer function
freq_f, H_f = signal.sosfreqz(sos_f, worN=4100, fs=fs)

# Group delay
freqg_f, gd_f = signal.group_delay((coefficients, 1), 512, fs=fs)

# Plotting
fig = plt.figure(figsize=(8, 7))

# Subplot 1
ax1 = fig.add_subplot(2, 2, 1)
ax1.semilogx(freq_f, 20 * np.log10(np.maximum(abs(H_f), 1e-5)))
ax1.set_xlabel('Frequency [Hz]')
ax1.set_ylabel(r'$|H(j \omega)|$ [dB]')
ax1.axis((10, fs / 2, -100, 10))
ax1.grid(which='both', axis='both')

# Subplot 2
ax2 = fig.add_subplot(2, 2, 2)
ax2.semilogx(freq_f, np.rad2deg(np.unwrap(np.angle(H_f))))
ax2.set_xlabel('Frequency [Hz]')
ax2.set_ylabel(r'$\angle H(j \omega)$ [$^\circ$]')
#ax2.axis((10, fs / 2, 100, 4000))
ax2.set_xlim(100,10000)
ax2.grid(which='both', axis='both')

# Subplot 3
ax3 = fig.add_subplot(2, 2, 3)
ax3.semilogx(freqg_f, 1000 * gd_f / fs)
ax3.set_xlabel('Frequency [Hz]')
ax3.set_ylabel(r'$-\frac{d\angle H(j \omega)}{d \omega}$ [ms]')
#ax3.axis((10, fs / 2, 0, 0.3))
ax3.set_xlim(100,4000)
ax3.set_ylim(0,3)
ax3.grid(which='both', axis='both')

# Subplot 4
ax4 = fig.add_subplot(2, 2, 4)
ax4.plot(np.linspace(0, (len(coefficients)-1)/fs, len(coefficients)), coefficients)
ax4.set_xlabel('Time [s]')
ax4.set_ylabel(r'$h(t)$')
ax4.grid(which='both', axis='both')
plt.tight_layout()
plt.show()


In [ ]:
#Filtrar com FIR
xn_fir_filtered = signal.sosfilt(sos_f, xn)

plt.figure(figsize=(12,3))
plt.plot(time_xn, xn_fir_filtered, linewidth=1, alpha=0.8,
         label='Filtered Signal')
plt.legend(loc='upper right')
plt.grid(linestyle = '--', which='both')
plt.xlabel('Tempo [s]')
plt.ylabel(r'$x(t)$ [-]')
plt.xlim(0, time_xn[-1])
plt.tight_layout()

plt.figure(figsize=(12,3))
plt.plot(time_xn, xn, '-k', linewidth=1, alpha=0.8,
         label='Original Signal')
plt.legend(loc='upper right')
plt.grid(linestyle = '--', which='both')
plt.xlabel('Tempo [s]')
plt.ylabel(r'$x(t)$ [-]')
plt.xlim(0, time_xn[-1])
plt.tight_layout()

ipd.Audio(xn_fir_filtered, rate=fs)

In [ ]:
from tqdm.notebook import trange, tqdm

#FIR FIlter here... !!!
def our_filter(xn, b, a):
    
    #length of xn
    N = len(xn)
    #order
    od = len(b)
    #initialize yn
    yn = np.zeros(N)
    #loop
    #bar = tqdm(total=N, desc= 'Filtering ...')
    
    for jn in np.arange(od, N-od): #range(od, N):
        x_slice = np.flip(xn[jn - od : jn])
        y_slice = np.flip(yn[jn - od + 1 : jn+1])
        yn[jn] = np.dot(b, x_slice) - np.dot(a, y_slice)
        #bar.update(1)
    #bar.close()
    gain = np.std(xn) / np.std(yn)
        
    #bar.close()
    #gain = np.std(xn)/np.std(yn)
    return yn
    